In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import nengo
import nengolib

In [3]:
# The adaptation term is just a lowpass filtered version of the spikes

ens_seed = 0
dt = 0.001
t = 0.5

with nengo.Network() as model:
    u = nengo.Node(output=lambda t: np.sin(2*np.pi*t))
    x = nengo.Ensemble(10, 1, neuron_type=nengo.AdaptiveLIF(), seed=ens_seed)

    nengo.Connection(u, x, synapse=None)

    recurrent_synapse = (x.neuron_type.inc_n * nengolib.signal.cont2discrete(
        nengo.Lowpass(x.neuron_type.tau_n), dt=dt, method='euler'))

    p_actual = nengo.Probe(x.neurons, 'adaptation', synapse=~nengolib.signal.z)
    p_expected = nengo.Probe(x.neurons, 'spikes', synapse=recurrent_synapse)
    
with nengo.Simulator(model, dt=dt) as sim:
    sim.run(t)

assert np.allclose(sim.data[p_actual], sim.data[p_expected])

Building finished in 0:00:01.                                                   


c:\users\aaron\ctn\nengolib\nengolib\signal\system.py:197: UserWarning: y0 (None!=0) does not properly initialize the system; see Nengo issue #1124.
  "Nengo issue #1124." % y0, UserWarning)


Simulating finished in 0:00:01.                                                 


In [4]:
# Since the terms are identical, we can simply feed
# these filtered spikes back to cancel out the adaptation term!

def install_tuning_curves(ens):
    """Use ens.seed to set ens.{max_rates, intercepts, gain, bias}."""
    rng = np.random.RandomState(ens.seed)
    ens.max_rates = ens.max_rates.sample(ens.n_neurons, rng=rng)
    ens.intercepts = ens.intercepts.sample(ens.n_neurons, rng=rng)
    ens.gain, ens.bias = ens.neuron_type.gain_bias(
        ens.max_rates, ens.intercepts)

def go(neuron_type, ens_seed=0, dt=0.001, t=0.01):
    with nengo.Network() as model:
        u = nengo.Node(output=lambda t: np.sin(2*np.pi*5*t))
        x = nengo.Ensemble(500, 1, neuron_type=neuron_type, seed=ens_seed)
        install_tuning_curves(x)        

        nengo.Connection(u, x, synapse=None)
        if isinstance(neuron_type, nengo.AdaptiveLIFRate):
            recurrent_synapse = (x.neuron_type.inc_n * nengolib.signal.cont2discrete(
                nengo.Lowpass(x.neuron_type.tau_n), dt=dt, method='euler'))
            nengo.Connection(x.neurons, x.neurons, transform=1/x.gain, synapse=recurrent_synapse)

        p_x = nengo.Probe(x, synapse=None)

    with nengo.Simulator(model, dt=dt) as sim:
        sim.run(t)
        
    return sim.data[p_x]

In [5]:
runs = (  # Note: These will have the same gains/biases
    go(nengo.AdaptiveLIF(tau_n=0.5, inc_n=0.1)),
    go(nengo.LIF()),
)

Building finished in 0:00:01.                                                   
Simulating finished in 0:00:01.                                                 
Building finished in 0:00:01.                                                   
Simulating finished in 0:00:01.                                                 


In [6]:
assert np.allclose(runs[0], runs[1])

In [7]:
# Conversely, adding this connection to a LIF population will effectively
# convert it into an aLIF population! (not shown because it is "obvious")